This notebook is meant to reproduce the unrecoverable OOM error thrown in the Apache Ignite system as a result of IGNITE-12042.

# Configuration

In [ ]:
%%bash

# Change these paths according to what you need. The default should work on a vanilla Chameleon Cloud instance.
IGNITE_DIR="/home/cc/apache-ignite"
DATA_DIR="/home/cc"

# Install dependencies

In [ ]:
%%bash

# Install Java 8 and Maven
sudo apt update
sudo apt install openjdk-8-jdk
sudo apt install maven

# Install Ignite

## Clone repository

In [ ]:
%%bash

# Clone the repository and use ignite-12042 (patch) branch
git clone https://github.com/gridgain/apache-ignite.git
cd /home/cc/apache-ignite
git reset --hard origin/ignite-12042

# Generate pre-patch data

## Revert patch

In [ ]:
%%bash

# Revert the fix (line 105 of RowStore.java)
sed -i "105d" /home/cc/apache-ignite/modules/core/src/main/java/org/apache/ignite/internal/processors/cache/persistence/RowStore.java

## Add logging to test

In [ ]:
%%bash

# Add logging
FILE="$(IGNITE_DIR)/modules/core/src/test/java/org/apache/ignite/internal/processors/cache/CacheIgniteOutOfMemoryExceptionTest.java"
sed -i "35  i\import java.io.FileWriter;\nimport java.io.PrintWriter;\nimport java.io.IOException;" $FILE
sed -i "131 i\        int key = 0;" $FILE
sed -i "134 d" $FILE
sed -i "134 i\                for (; key < 500_000; ++key)" $FILE
sed -i "140 i\                try (FileWriter fw = new FileWriter(\"$(DATA_DIR)/pre-patch.txt\", true); PrintWriter writer = new PrintWriter(fw)) {\n                    writer.printf(\"%d\\n\", key);\n                } catch (IOException xxx) {}" $FILE

## Run failing test

This will generate `pre-patch.txt` in `/home/cc/`

In [ ]:
%%bash

# Run the failing test
cd /home/cc/apache-ignite/modules/core
mvn test -Dtest=CacheIgniteOutOfMemoryExceptionTest

# Generate post-patch data

## Apply patch

In [ ]:
%%bash

# Apply the fix
git reset --hard origin/ignite-12042

## Add logging to test

In [ ]:
%%bash

# Add logging
FILE="$(IGNITE_DIR)/apache-ignite/modules/core/src/test/java/org/apache/ignite/internal/processors/cache/CacheIgniteOutOfMemoryExceptionTest.java"
sed -i "35  i\import java.io.FileWriter;\nimport java.io.PrintWriter;\nimport java.io.IOException;" $FILE
sed -i "131 i\        int key = 0;" $FILE
sed -i "134 d" $FILE
sed -i "134 i\                for (; key < 500_000; ++key)" $FILE
sed -i "140 i\                try (FileWriter fw = new FileWriter(\"$(DATA_DIR)/post-patch.txt\", true); PrintWriter writer = new PrintWriter(fw)) {\n                    writer.printf(\"%d\\n\", key);\n                } catch (IOException xxx) {}" $FILE

## Run successful test

In [ ]:
%%bash

# Run the successful test
mvn test -Dtest=CacheIgniteOutOfMemoryExceptionTest

# Create plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
prepatch = []
postpatch = []

with open("{}/pre-patch.txt".format(os.environ["DATA_DIR"])) as file:
    for line in file:
        prepatch.append(int(line))

with open("{}/post-patch.txt".format(os.environ["DATA_DIR"])) as file:
    for line in file:
        postpatch.append(int(line))

# Pad prepatch retry array with 0s (lack of retries ~ failed retries).
length = len(postpatch)
indices = np.arange(length)
prepatch = prepatch + (length - len(prepatch)) * [0]

width = 0.3
plt.bar(indices, prepatch, width, label='pre-patch')
plt.bar(indices + width, postpatch, width, label='post-patch')

plt.ylabel('Successful puts < OOM thrown')
plt.xlabel('Retry ID')
plt.title('IGNITE-12042: Unrecoverable OOM')

plt.legend(loc='best')
plt.show()